In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime    

In [2]:
# ---------------------------- Logging Function ----------------------------
def log_progress(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("code_log.txt", "a") as f:
        f.write(f"[{timestamp}] {message}\n")

In [3]:
# ---------------------------- Extract Function ----------------------------
def extract(url, table_attribs):
    log_progress("Sending HTTP request to data source.")
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    tables = soup.find_all('table')

    for i, table in enumerate(tables):
        headers = [th.get_text(strip=True) for th in table.find_all('th')]
        if "Market cap(US$ billion)" in headers:
            log_progress(f"Found correct table at index {i}")
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 3 and cols[1].find('a'):
                    bank_name = cols[1].a.text.strip()
                    market_cap = cols[2].text.strip().replace(',', '')
                    try:
                        market_cap = float(market_cap)
                        df1 = pd.DataFrame([[bank_name, market_cap]], columns=table_attribs)
                        df = pd.concat([df, df1], ignore_index=True)
                    except ValueError:
                        continue
            break

    if df.empty:
        log_progress("No data extracted. Table structure may have changed.")

    return df

In [4]:
# ---------------------------- Transform Function ----------------------------
def transform(df, csv_path):
    exchange_rates = pd.read_csv(csv_path)
    exchange_rates.columns = exchange_rates.columns.str.strip()
    rates = exchange_rates.set_index('Currency')['Rate'].to_dict()

    df.columns = df.columns.str.strip()
    df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(float)
    df['MC_GBP_Billion'] = df['MC_USD_Billion'].apply(lambda x: round(x * rates['GBP'], 2))
    df['MC_EUR_Billion'] = df['MC_USD_Billion'].apply(lambda x: round(x * rates['EUR'], 2))
    df['MC_INR_Billion'] = df['MC_USD_Billion'].apply(lambda x: round(x * rates['INR'], 2))
    return df

In [5]:
# ---------------------------- Load Functions ----------------------------
def load_to_csv(df, output_path):
    df.to_csv(output_path, index=False)

def load_to_db(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [6]:
# ---------------------------- Main Execution ----------------------------
if __name__ == "__main__":
    data_url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
    exchange_csv_path = "./exchange_rate.csv"
    output_csv_path = "./Largest_banks_data.csv"
    db_name = "Banks.db"
    table_name = "Largest_banks"

    log_progress("ETL Job Started")

    # Extract
    log_progress("Starting data extraction")
    table_attribs = ["Name", "MC_USD_Billion"]
    df_extracted = extract(data_url, table_attribs)
    log_progress("Data extraction completed")

    # Transform
    log_progress("Starting data transformation")
    df_transformed = transform(df_extracted, exchange_csv_path)
    log_progress("Data transformation completed")

    # Load
    log_progress("Saving to CSV")
    load_to_csv(df_transformed, output_csv_path)
    log_progress("CSV file saved")

    log_progress("Saving to database")
    load_to_db(df_transformed, db_name, table_name)
    log_progress("Data saved to database")

    log_progress("ETL Job Completed")
    sql_connection = sqlite3.connect('Banks.db')
    log_progress('SQL Connection initiated.')
    log_progress('Data loaded to Database as table. Running the query')
    
    query_statement = f"SELECT * FROM {table_name}"
    query_statement2 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
    run_query(query_statement, sql_connection)
    run_query(query_statement2, sql_connection)

SELECT * FROM Largest_banks
  Name  MC_USD_Billion  MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion
0               432.92          346.34          402.62        35910.71
1               231.52          185.22          215.31        19204.58
2               194.56          155.65          180.94        16138.75
3               160.68          128.54          149.43        13328.41
4               157.91          126.33          146.86        13098.63
5               155.87          124.70          144.96        12929.42
6               148.90          119.12          138.48        12351.26
7               140.83          112.66          130.97        11681.85
8               139.82          111.86          130.03        11598.07
9               136.81          109.45          127.23        11348.39
SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


C:\Users\deepa\AppData\Local\Temp\ipykernel_2276\1576424448.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1], ignore_index=True)
